In [ ]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm

# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [ ]:
tool_change_pose = ....

In [ ]:
rospy.init_node('FPD_demo')

In [ ]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
#pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()
    
    
def set_cutting_goal(pointA, pointB):
    material= MaterialType()
    material = material.PLASTIC

    normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
    normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")

    goal=CuttingGoal()
    goal.surface_normal_start=normalA
    goal.surface_normal_end=normalB
    goal.start_location=pointA
    goal.end_location=pointB
    goal.material.material_type=material
    goal.timeout=5000
    goal.thickness = 0.005 #not important. can be anything
    
def set_screwing_goal(B_normal, B_screw_guess):
    sc_goal = UnscrewGoal()
    sc_goal.screw_location.header.frame_id = "hrr_cobot.base_link"
    sc_goal.surface_normal.header.frame_id = "hrr_cobot.base_link"
    sc_goal.surface_normal.vector = hrr_common.ros_utils.np2vec3(B_normal)
    sc_goal.screw_location.point = hrr_common.ros_utils.np2vec3(B_screw_guess)
    sc_goal.contact_force = 5
    sc_goal.insertion_force = 10
    sc_goal.timeout = 1200.0
    
    
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)

### Load robot

In [ ]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
#cobot.change_tool("nothing")

In [ ]:
#!rosparam get /hrr_cobot/tool_name

# PC Tower disassembly

## Tool Change to Screwdriver

## Unscrew screw (eight times)

## Change tool from screwdriver to grinder 

## Cut the gray panel

## Change tool to vacuum griper

## Pick up the screen/grey/whatever

## Change the tool to scew driver

## Unscrew screw (to be called 4 times)

In [ ]:
## 